## Autoencoder based recommendation model

Basic structure for autoencoder based recommendation model, to fill the sparse rating matrix with predicted ratings. When doing evaluation on test set, only the existing user-item pairs will be factored in the loss while the prediction result will be a full matrix with all the ratings, from there we can sort items to user and make top-k recommendations. 

This notebook includes:
1. Data preparation and methods in splitting datasets for recommendation train and test process.
2. Structure of autoencoder.
3. Search for hyperparameters and simple experiments on model structure.

In [51]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import sys
#insert search path for packages
sys.path.insert(0, '../common/')
sys.path.insert(0, '../models/')
from importlib import reload

#project scripts
import util as util
import preprocessor as prep
import encoder as EN
reload(prep)
reload(util)
reload(EN)

<module 'encoder' from '../models/encoder.py'>

## Preprocess data

In [3]:
ratings = pd.read_csv('../dataset/ml-1m/ratings.dat', delimiter = '::',header=None, engine='python')
ratings.columns = ['userid','itemid','rating','timestamp']
ratings_reindex = prep.reindexer(ratings,'userid','itemid','rating')

In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(ratings_reindex,
                               stratify=ratings_reindex['encoded_users'],
                               test_size=0.1,
                               random_state=42)

In [5]:
training_set = np.array(train, dtype = 'int')
test_set = np.array(test, dtype = 'int')

In [6]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [7]:
# creating matrix form of ratings
def convert(data):
    new_data = []
    for id_users in range(nb_users+1):
        # each user's watched movies
        # data[:,0], first column, all rows column users
        id_items = data[:,1][data[:,0] == id_users]
        # each user's rating for that item
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        # the positions of these items are filled with ratings, creating the matrix
        ratings[id_items-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [8]:
%%time
training_set = convert(training_set)
test_set = convert(test_set)

CPU times: user 9.2 s, sys: 417 ms, total: 9.61 s
Wall time: 9.62 s


In [9]:
# converting to tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Automated creation of encoder structure

Create autoencoder network structure based on a list
containing the units

e.g. 
[input_size,20,10] means there are 2 + 2 layers in total
(len(L)-1 * 2) number of layers in total

In [52]:
autoencoder_network = EN.Encoder([nb_movies,20,10],'sigmoid',0.1)
criterion = nn.MSELoss()
optimizer = optim.RMSprop(autoencoder_network.parameters(), lr = 0.01, weight_decay = 0.5)

## Training encoder


In [ ]:
nb_epoch = 20
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    # s is the number of users who rated at least 1 movies
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = autoencoder_network(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) #making this anyway not equal to 0, as this will be a denominator
            #mean_corrector is the avg of the error, only considering the movies having ratings (non-zero ratings) for computing mean of error
            loss.backward() # decide the direction the increment of weights
            #this call will just computing all the gradients required
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step() # decide the amount to update the weights
            
    print('epoch: '+str(epoch)+' loss: '+ str(train_loss/s))

## Testing encoder


In [142]:
test_loss = 0
s = 0.
# averaged difference between real rating and predicted rating
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0) # should keep the training set
    target = Variable(test_set[id_user]).unsqueeze(0) # to predict the other movies user not seen yet
    
    if torch.sum(target.data > 0) > 0:
        # make predictions
        output = autoencoder_network(input)
        target.require_grad = False
        output[target == 0] = 0 # dont want to measue the loss on the movies didnt get the actual rating from user 
        # force to 0 and difference / loss will be 0 for those entries
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) 
        # only consider the movies that are rated in the test set, to be included in the loss
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9552)
